In [1]:
import os
os.chdir('C:/Users/emrijai/Documents/IPython Notebooks/MS7331/Project1/MSDS7331_Project1')
os.getcwd()

'C:\\Users\\emrijai\\Documents\\IPython Notebooks\\MS7331\\Project1\\MSDS7331_Project1'

In [2]:
import pandas as pd
import numpy as np
import copy
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.style.use('ggplot')

import warnings
warnings.simplefilter('ignore', DeprecationWarning)
warnings.simplefilter('ignore', FutureWarning)

from pandas.tools.plotting import scatter_matrix

#Bring in data set
loan_df = pd.read_csv('LoanData/loanData.csv')

C:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Remove unwanted attributes from the data set based on specific criteria
for col in ['id','funded_amnt_inv','sub_grade','emp_title','pymnt_plan','url','title','zip_code','earliest_cr_line',
            'mths_since_last_delinq','mths_since_last_record','pub_rec','revol_bal','revol_util','total_acc',
            'initial_list_status','out_prncp_inv','total_pymnt_inv','recoveries','collection_recovery_fee','last_pymnt_d',
            'last_pymnt_amnt','next_pymnt_d','last_credit_pull_d','collections_12_mths_ex_med','policy_code','application_type',
            'annual_inc_joint','dti_joint','verification_status_joint','open_acc_6m','open_il_6m','open_il_12m','open_il_24m',
            'mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util',
            'total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m', 'installment', 'mths_since_last_major_derog']:
    if col in loan_df:
        del loan_df[col]

In [4]:
#Modify term to be integer for easier analysis
loan_df = loan_df.replace(to_replace=" 36 months", value=36)
loan_df = loan_df.replace(to_replace=" 60 months", value=60)

#Impute missing data values with median
for col in ['annual_inc', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'acc_now_delinq']:
    df_temp = copy.deepcopy(loan_df[col])
    df_temp = df_temp.replace(to_replace=np.nan, value=df_temp.median())
    loan_df[col] = df_temp

Loan Status is divided into 3 classes of loans - Current, Bad and Good. The idea is to use the 'Bad' and 'Good' cases to train a model. The training data set from this data subset (Good+Bad) is used for train/test/validation. The reminaing data i.e. cases under status 'Current', can then be fed into the validated model to predict which loans can go 'Bad'/'Good', based on the features we have of the customers.
Bad Loans are coded = 0
Good Loans are coded = 1
Current loans are separated out into new data set for predictions later on

In [5]:
#Create new variables for classification
#Nominal classification of loan_status
loan_df.loc[loan_df['loan_status'] == 'Charged Off' , 'Loan_Class'] = "Bad"
loan_df.loc[loan_df['loan_status'] == 'Default' , 'Loan_Class'] = "Bad"
loan_df.loc[loan_df['loan_status'] == 'Late (31-120 days)' , 'Loan_Class'] = "Bad"
loan_df.loc[loan_df['loan_status'] == 'Late (16-30 days)' , 'Loan_Class'] = "Bad"
loan_df.loc[loan_df['loan_status'] == 'Does not meet the credit policy. Status:Charged Off' , 'Loan_Class'] = "Bad"

loan_df.loc[loan_df['loan_status'] == 'Fully Paid' , 'Loan_Class'] = "Good"
loan_df.loc[loan_df['loan_status'] == 'Does not meet the credit policy. Status:Fully Paid' , 'Loan_Class'] = "Good"

loan_df.loc[loan_df['loan_status'] == 'Current' , 'Loan_Class'] = "Current"
loan_df.loc[loan_df['loan_status'] == 'In Grace Period' , 'Loan_Class'] = "Current"
loan_df.loc[loan_df['loan_status'] == 'Issued' , 'Loan_Class'] = "Current"

#Numerical classification of grade
loan_df.loc[loan_df['grade'] == 'A', 'Grade_Class'] = 1
loan_df.loc[loan_df['grade'] == 'B', 'Grade_Class'] = 2
loan_df.loc[loan_df['grade'] == 'C', 'Grade_Class'] = 3
loan_df.loc[loan_df['grade'] == 'D', 'Grade_Class'] = 4
loan_df.loc[loan_df['grade'] == 'E', 'Grade_Class'] = 5
loan_df.loc[loan_df['grade'] == 'F', 'Grade_Class'] = 6
loan_df.loc[loan_df['grade'] == 'G', 'Grade_Class'] = 7

#Numerical classification of employment length
loan_df.loc[loan_df['emp_length'] == '< 1 year', 'Emp_Class'] = 0
loan_df.loc[loan_df['emp_length'] == '1 year', 'Emp_Class'] = 1
loan_df.loc[loan_df['emp_length'] == '2 years', 'Emp_Class'] = 2
loan_df.loc[loan_df['emp_length'] == '3 years', 'Emp_Class'] = 3
loan_df.loc[loan_df['emp_length'] == '4 years', 'Emp_Class'] = 4
loan_df.loc[loan_df['emp_length'] == '5 years', 'Emp_Class'] = 5
loan_df.loc[loan_df['emp_length'] == '6 years', 'Emp_Class'] = 6
loan_df.loc[loan_df['emp_length'] == '7 years', 'Emp_Class'] = 7
loan_df.loc[loan_df['emp_length'] == '8 years', 'Emp_Class'] = 8
loan_df.loc[loan_df['emp_length'] == '9 years', 'Emp_Class'] = 9
loan_df.loc[loan_df['emp_length'] == '10+ years', 'Emp_Class'] = 10
loan_df.loc[loan_df['emp_length'] == 'n/a', 'Emp_Class'] = 0

#Nominal classification of home_ownership
loan_df.loc[loan_df['home_ownership'] == 'NONE', 'home_ownership_class'] = 'OTHER'
loan_df.loc[loan_df['home_ownership'] == 'OTHER', 'home_ownership_class'] = 'OTHER'
loan_df.loc[loan_df['home_ownership'] == 'ANY', 'home_ownership_class'] = 'OTHER'
loan_df.loc[loan_df['home_ownership'] == 'MORTGAGE', 'home_ownership_class'] = 'MORTGAGE'
loan_df.loc[loan_df['home_ownership'] == 'RENT', 'home_ownership_class'] = 'RENT'
loan_df.loc[loan_df['home_ownership'] == 'OWN', 'home_ownership_class'] = 'OWN'

#Nominal classification of verification_status
loan_df.loc[loan_df['verification_status'] == 'Verified', 'verification_status_class'] = 'V'
loan_df.loc[loan_df['verification_status'] == 'Source Verified', 'verification_status_class'] = 'SV'
loan_df.loc[loan_df['verification_status'] == 'Not Verified', 'verification_status_class'] = 'NV'

#Range of interest rates
loan_df['Int_Rate_Range'] = pd.cut(loan_df.int_rate, [0, 5,10,15,20,25, 30], 6)
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 33 columns):
member_id                    887379 non-null int64
loan_amnt                    887379 non-null float64
funded_amnt                  887379 non-null float64
term                         887379 non-null int64
int_rate                     887379 non-null float64
grade                        887379 non-null object
emp_length                   887379 non-null object
home_ownership               887379 non-null object
annual_inc                   887379 non-null float64
verification_status          887379 non-null object
issue_d                      887379 non-null object
loan_status                  887379 non-null object
desc                         126029 non-null object
purpose                      887379 non-null object
addr_state                   887379 non-null object
dti                          887379 non-null float64
delinq_2yrs                  887379 non-null float64
i

In [32]:
Int_Rate_Range = loan_df.groupby(by="Int_Rate_Range")
Int_Rate_Range.loan_amnt.count()

Int_Rate_Range
(0, 5]           0
(5, 10]     235621
(10, 15]    372060
(15, 20]    221068
(20, 25]     52137
(25, 30]      6493
Name: loan_amnt, dtype: int64

In [7]:
categorical = ["Loan_Class", "term", "home_ownership", "verification_status", "purpose", "addr_state", "Int_Rate_Range", "home_ownership_class","verification_status_class" ]

continuous = ["loan_amnt", "funded_amnt", "int_rate", "annual_inc", "dti", "total_pymnt", "total_rec_prncp", "total_rec_int",
             "total_rec_late_fee", "tot_coll_amt", "tot_cur_bal"]

ordinal = ["Grade_Class", "Emp_Class"]

loan_df[continuous] = loan_df[continuous].astype(np.float64)
loan_df[ordinal] = loan_df[ordinal].astype(np.int64)

loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 33 columns):
member_id                    887379 non-null int64
loan_amnt                    887379 non-null float64
funded_amnt                  887379 non-null float64
term                         887379 non-null int64
int_rate                     887379 non-null float64
grade                        887379 non-null object
emp_length                   887379 non-null object
home_ownership               887379 non-null object
annual_inc                   887379 non-null float64
verification_status          887379 non-null object
issue_d                      887379 non-null object
loan_status                  887379 non-null object
desc                         126029 non-null object
purpose                      887379 non-null object
addr_state                   887379 non-null object
dti                          887379 non-null float64
delinq_2yrs                  887379 non-null float64
i

In [8]:
#Creating the copy of the dataset with all transformed the variables retained
loan_df_copy = loan_df.copy()

In [9]:
loan_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 33 columns):
member_id                    887379 non-null int64
loan_amnt                    887379 non-null float64
funded_amnt                  887379 non-null float64
term                         887379 non-null int64
int_rate                     887379 non-null float64
grade                        887379 non-null object
emp_length                   887379 non-null object
home_ownership               887379 non-null object
annual_inc                   887379 non-null float64
verification_status          887379 non-null object
issue_d                      887379 non-null object
loan_status                  887379 non-null object
desc                         126029 non-null object
purpose                      887379 non-null object
addr_state                   887379 non-null object
dti                          887379 non-null float64
delinq_2yrs                  887379 non-null float64
i

In [10]:
# Now let's clean up the dataset. We don't need features for now.
if 'member_id' in loan_df:
    del loan_df['member_id'] # if 'member_id' column still exists, delete it. We don't need this for feature for now.
    
if 'emp_length' in loan_df:
    del loan_df['emp_length']
    
if 'verification_status' in loan_df:
    del loan_df['verification_status']
    
if 'home_ownership' in loan_df:
    del loan_df['home_ownership']
    
if 'issue_d' in loan_df:
    del loan_df['issue_d']

if 'desc' in loan_df:
    del loan_df['desc']

if 'addr_state' in loan_df:
    del loan_df['addr_state']

if 'grade' in loan_df:
    del loan_df['grade']

if 'loan_status' in loan_df:
    del loan_df['loan_status']

In [11]:
#Stats of nominal and continous data
loan_df.head()

,loan_amnt,funded_amnt,term,int_rate,annual_inc,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,...,total_rec_late_fee,acc_now_delinq,tot_coll_amt,tot_cur_bal,Loan_Class,Grade_Class,Emp_Class,home_ownership_class,verification_status_class,Int_Rate_Range
0,5000.0,5000.0,36,10.65,24000.0,credit_card,27.65,0.0,1.0,3.0,...,0.00,0.0,NaN,NaN,Good,2,10,RENT,V,"(10, 15]"
1,2500.0,2500.0,60,15.27,30000.0,car,1.00,0.0,5.0,3.0,...,0.00,0.0,NaN,NaN,Bad,3,0,RENT,SV,"(15, 20]"
2,2400.0,2400.0,36,15.96,12252.0,small_business,8.72,0.0,2.0,2.0,...,0.00,0.0,NaN,NaN,Good,3,10,RENT,NV,"(15, 20]"
3,10000.0,10000.0,36,13.49,49200.0,other,20.00,0.0,1.0,10.0,...,16.97,0.0,NaN,NaN,Good,3,10,RENT,SV,"(10, 15]"
4,3000.0,3000.0,60,12.69,80000.0,other,17.94,0.0,0.0,15.0,...,0.00,0.0,NaN,NaN,Current,2,1,RENT,SV,"(10, 15]"


In [12]:
loan_df_copy.head()

,member_id,loan_amnt,funded_amnt,term,int_rate,grade,emp_length,home_ownership,annual_inc,verification_status,...,total_rec_late_fee,acc_now_delinq,tot_coll_amt,tot_cur_bal,Loan_Class,Grade_Class,Emp_Class,home_ownership_class,verification_status_class,Int_Rate_Range
0,1296599,5000.0,5000.0,36,10.65,B,10+ years,RENT,24000.0,Verified,...,0.00,0.0,NaN,NaN,Good,2,10,RENT,V,"(10, 15]"
1,1314167,2500.0,2500.0,60,15.27,C,< 1 year,RENT,30000.0,Source Verified,...,0.00,0.0,NaN,NaN,Bad,3,0,RENT,SV,"(15, 20]"
2,1313524,2400.0,2400.0,36,15.96,C,10+ years,RENT,12252.0,Not Verified,...,0.00,0.0,NaN,NaN,Good,3,10,RENT,NV,"(15, 20]"
3,1277178,10000.0,10000.0,36,13.49,C,10+ years,RENT,49200.0,Source Verified,...,16.97,0.0,NaN,NaN,Good,3,10,RENT,SV,"(10, 15]"
4,1311748,3000.0,3000.0,60,12.69,B,1 year,RENT,80000.0,Source Verified,...,0.00,0.0,NaN,NaN,Current,2,1,RENT,SV,"(10, 15]"


In [13]:
# perform one-hot encoding of the categorical data "embarked"
tmp_df = pd.get_dummies(loan_df.home_ownership_class,prefix='HOC')
loan_df = pd.concat((loan_df,tmp_df),axis=1) # add back into the dataframe

tmp_df = pd.get_dummies(loan_df.verification_status_class,prefix='VSC')
loan_df = pd.concat((loan_df,tmp_df),axis=1) # add back into the dataframe

tmp_df = pd.get_dummies(loan_df.purpose,prefix='Loan_Purpose')
loan_df = pd.concat((loan_df,tmp_df),axis=1) # add back into the dataframe

In [14]:
loan_df[['home_ownership_class','verification_status_class', 'purpose', 'Loan_Class', 'Int_Rate_Range']].describe().transpose()

,count,unique,top,freq
home_ownership_class,887379,4,MORTGAGE,443557
verification_status_class,887379,3,SV,329558
purpose,887379,14,debt_consolidation,524215
Loan_Class,887379,3,Current,616492
Int_Rate_Range,887379,5,"(10, 15]",372060


In [15]:
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 45 columns):
loan_amnt                          887379 non-null float64
funded_amnt                        887379 non-null float64
term                               887379 non-null int64
int_rate                           887379 non-null float64
annual_inc                         887379 non-null float64
purpose                            887379 non-null object
dti                                887379 non-null float64
delinq_2yrs                        887379 non-null float64
inq_last_6mths                     887379 non-null float64
open_acc                           887379 non-null float64
out_prncp                          887379 non-null float64
total_pymnt                        887379 non-null float64
total_rec_prncp                    887379 non-null float64
total_rec_int                      887379 non-null float64
total_rec_late_fee                 887379 non-null float64
acc_now

In [16]:
#Remove the redundant variables
if 'home_ownership_class' in loan_df:
    del loan_df['home_ownership_class']
    
if 'verification_status_class' in loan_df:
    del loan_df['verification_status_class']
    
if 'purpose' in loan_df:
    del loan_df['purpose']

In [17]:
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 42 columns):
loan_amnt                          887379 non-null float64
funded_amnt                        887379 non-null float64
term                               887379 non-null int64
int_rate                           887379 non-null float64
annual_inc                         887379 non-null float64
dti                                887379 non-null float64
delinq_2yrs                        887379 non-null float64
inq_last_6mths                     887379 non-null float64
open_acc                           887379 non-null float64
out_prncp                          887379 non-null float64
total_pymnt                        887379 non-null float64
total_rec_prncp                    887379 non-null float64
total_rec_int                      887379 non-null float64
total_rec_late_fee                 887379 non-null float64
acc_now_delinq                     887379 non-null float64
tot_co

In [37]:
loan_df_Current = loan_df[loan_df['Loan_Class'] == 'Current']
valuelist = ['Bad', 'Good']
loan_df_Historical = loan_df[loan.column.isin(value_list)]

loan_df_Historical = (loan_df[loan_df['Loan_Class'] == 'Bad'] | loan_df[loan_df['Loan_Class'] == 'Good'])

NotImplementedError: couldn't find matching opcode for 'or_bdd'

In [36]:
loan_df_Historical['Loan_Class'].head()

4     Current
6     Current
32    Current
39    Current
61    Current
Name: Loan_Class, dtype: object